In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.decomposition import NMF
from sklearn.metrics import ndcg_score, average_precision_score
from sklearn.decomposition import NMF, TruncatedSVD

In [2]:
# ALS class 정의
# http://ethen8181.github.io/machine-learning/recsys/1_ALSWR.html

class ALS:
    # 하이퍼 파라미터 지정
    def __init__(self, factors=10, iterations=20, reg=0.01):
        self.factors = factors
        self.iterations = iterations
        self.reg = reg
    # 모델 적합 -> 평점 행렬 입력
    def fit(self, ratings):
        # 랜덤으로 user 수 * latent factor 형태의 행렬 생성
        self.user_factors = np.random.random((ratings.shape[0], self.factors))
        # 랜덤으로 item 수 * latent factor 형태의 행렬 생성
        self.item_factors = np.random.random((ratings.shape[1], self.factors))
        
        # 사전에 지정한 iteration 수에 걸쳐서, 교차로 als_step 진행
        for _ in range(self.iterations):
            # user_factors 먼저 업데이트 
            self.user_factors = self.als_step(ratings, self.user_factors, self.item_factors)
            # 이어서 item_factors 업데이트
            self.item_factors = self.als_step(ratings.T, self.item_factors, self.user_factors)
    
    # 교차로 업데이트하는 스텝 메서드
    def als_step(self, ratings, solve_vecs, fixed_vecs):
        # normal equation - 업데이트 되지 않을 user/item feature의 공분산 matrix
        # feature가 주어진(고정된) 상태에서 최적의 해를 찾아 그 행렬을 새로운 factors로 사용
        # 가령, user_factors가 고정되어 있을 때는 최적의 item_factors를 구하고, 반대도 마찬가지
        A = fixed_vecs.T.dot(fixed_vecs) + np.eye(self.factors) * self.reg
        b = ratings.dot(fixed_vecs)
        A_inv = np.linalg.inv(A)
        solve_vecs = b.dot(A_inv)
        return solve_vecs

    def predict(self):
        pred = self.user_factors.dot(self.item_factors.T)
        return pred

In [11]:
# 1. 데이터 로드
# url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx'
# path = '../../data/online_retail_II.xlsx'

data_path = "../../../data/online_retail/Online Retail.xlsx"
# df= pd.read_csv(data_path+'Online Retail.xlsx', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
df = pd.read_excel(data_path)

In [13]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [15]:
df['CustomerID'] = df['CustomerID'].astype('category')
df['StockCode'] = df['StockCode'].astype('category')
df = df.rename({"CustomerID":"CustomerID"}, axis=1)

In [16]:
# 2. 피벗 테이블 만들기 -> 너무 적은 인터렉션을 갖는 유저/아이템은 배제
interaction_counts = df.groupby('CustomerID').StockCode.count()
df = df[df.CustomerID.isin(interaction_counts[interaction_counts > 10].index)]

item_counts = df.StockCode.value_counts()
df = df[df.StockCode.isin(item_counts[item_counts > 10].index)]

In [17]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [18]:
pivot = df.pivot_table(index='CustomerID', columns='StockCode', fill_value=0, aggfunc='size')

In [19]:
# 3. implicit data로 변경 (binary화)
pivot = (pivot > 0).astype(int)

In [20]:
pivot.head()

StockCode,10002,10080,10120,10125,10133,10134,10135,11001,15030,15034,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [22]:
# 4. train/test split -> MF에서는 다른 방식!
test_ratio = 0.2
train = pivot.copy()
test = np.zeros(pivot.shape)

for user in range(pivot.shape[0]):
    test_interactions = np.random.choice(pivot.values[user, :].nonzero()[0], 
                                         size=int(test_ratio*np.sum(pivot.values[user, :])),
                                         replace=False)
    print(test_interactions)
    train.values[user, test_interactions] = 0.
    test[user, test_interactions] = pivot.values[user, test_interactions]
    print(test[user, test_interactions])

[]
[]
[2217   40 1897  761 3579 1069 3190 2079 1230 3391 1280 1458 3164 2300
 3424 1988 1651 1068 1905   12]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[2764  895 2766  722]
[1. 1. 1. 1.]
[1929 1930 2098  757  592 2052 1009 2494 2286 2337 2087  944 2081 2014]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[4062 3323 3276]
[1. 1. 1.]
[2167  674 1805  442 1916 4062  854 2168 1483 2059 1271]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[]
[]
[1677  592  334 1350 1877 1186 2020  396  970  833 1466]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1527 1271]
[1. 1.]
[ 292  377 2481  493  939 1777 1271 1283 1533  800]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1066 3042 2483  911  944 1206 2120 3311 1729 1996 2493  580  947  950
 1791 1271 1550  332   71 3313  391 2014 1647  311 1644 3697]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
[1483  943]
[1. 1.]
[3303  852 1744 1680 2221 1943  311  344 1686 1345 1561 1239 1872 1844
  115   92 1620 1463 2431   91  271 2334 2296 2209

In [8]:
# Convert train and test matrix into sparse matrix
train_csr = coo_matrix(train.values)
test_csr = coo_matrix(test)

In [12]:
n_latent_factors = 20

In [13]:
# SVD

# using sklearn Truncated SVD
svd = TruncatedSVD(n_components=n_latent_factors, random_state=42)
train_svd = svd.fit_transform(train_csr)
svd_pred = svd.inverse_transform(svd.transform(test_csr))

In [14]:
# using svds from scipy
u, sigma, vt = svds(train_csr.astype(float), n_latent_factors)
svd_pred = np.dot(u, np.dot(np.diag(sigma), vt))

In [15]:
print(f'shapes of the matrices: {u.shape, sigma.shape, vt.shape}')

shapes of the matrices: ((4383, 20), (20,), (20, 4632))


In [16]:
# NMF
model = NMF(n_components=n_latent_factors, init='random', random_state=0)

W = model.fit_transform(train_csr)
H = model.components_
nmf_pred = np.dot(W, H)

In [19]:
# train ALS model
als = ALS(factors = n_latent_factors, iterations=100, reg=0.01)
als.fit(train_csr)

# predict
als_pred = als.predict()

In [20]:
at_k = 10

# Make sure the predicted scores are in the range [0, 1]
predicted_svd = (svd_pred - svd_pred.min()) / (svd_pred.max() - svd_pred.min())
predicted_nmf = (nmf_pred - nmf_pred.min()) / (nmf_pred.max() - nmf_pred.min())
predicted_als = (als_pred - als_pred.min()) / (als_pred.max() - als_pred.min())

In [21]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE for SVD
svd_rmse = np.sqrt(mean_squared_error(test_csr.toarray(), predicted_svd))
print('SVD RMSE: ', svd_rmse)

# Calculate RMSE for NMF
nmf_rmse = np.sqrt(mean_squared_error(test_csr.toarray(), predicted_nmf))
print('NMF RMSE: ', nmf_rmse)

# Calculate RMSE for ALS
als_rmse = np.sqrt(mean_squared_error(test_csr.toarray(), predicted_als))
print('ALS RMSE: ', als_rmse)


SVD RMSE:  0.39042116410393973
NMF RMSE:  0.050480186367535824
ALS RMSE:  0.39910295781046595
